<h1>Kaggle: Favorita Grocery Sales Forecasting<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Descripción" data-toc-modified-id="Descripción-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Descripción</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Lectura-de-datos" data-toc-modified-id="Lectura-de-datos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Lectura de datos</a></span></li><li><span><a href="#Carga-y-preprocesado" data-toc-modified-id="Carga-y-preprocesado-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Carga y preprocesado</a></span></li><li><span><a href="#Muestras-de-entrenamiento-y-validación" data-toc-modified-id="Muestras-de-entrenamiento-y-validación-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Muestras de entrenamiento y validación</a></span></li><li><span><a href="#Entrenamiento-y-validación-del-modelo" data-toc-modified-id="Entrenamiento-y-validación-del-modelo-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Entrenamiento y validación del modelo</a></span></li></ul></div>

> Notebook para jugar con los datos de la competición de Kaggle [favorita-grocery-sales-forecasting](https://www.kaggle.com/c/favorita-grocery-sales-forecasting/)

# Descripción

"El objetivo del concurso es predecir las unidades vendidas de cada producto en una cadena de tiendas, por sede y fecha, durante las 2 semanas posteriores a los datos de entrenamiento."

* Descargas + Información completa sobre los datasets: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesRegressor
from IPython.display import display
from sklearn import metrics
import os

# Lectura de datos

Lo primero que hacemos antes de cargar el dataset, es ver qué contiene el fichero csv. Como hemos visto que es enorme (4,65GB) nos conviene optimizar el tamaño del dataframe, para lo que tendremos que ver qué columnas existen, sus tipos, y una muestra de los datos.

Hay varias formas de indagar lo que tiene el fichero csv:

In [3]:
!head -5 data/train.csv

id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,
1,2013-01-01,25,105574,1.0,
2,2013-01-01,25,105575,2.0,
3,2013-01-01,25,108079,1.0,


In [4]:
!tail -5 data/train.csv

125497035,2017-08-15,54,2089339,4.0,False
125497036,2017-08-15,54,2106464,1.0,True
125497037,2017-08-15,54,2110456,192.0,False
125497038,2017-08-15,54,2113914,198.0,True
125497039,2017-08-15,54,2116416,2.0,False


Vistos los datos, decidimos en qué tipo específico se puede encajar cada columna numérica, de forma que se ahorre espacio en memoria (Pandas no lo hace):

In [5]:
types = {'id': 'int32',
         'store_nbr': 'int8',
         'item_nbr': 'int32',
         'unit_sales': 'float32',
         'onpromotion': 'object'}

# Carga y preprocesado

Para poder realizar el preprocesado tenemos el hándicap de que no podemos hacerlo con el dataset completo de la forma habitual, ya que tendremos problemas de memoria con un PC de 16GB de RAM.
Existen varios enfoques:
 * Usar más RAM :P
 * Dividir los datos en chunks, hacer el preprocesado, y concatenar
 * Trabajar sólo con una parte del dataset original como primera táctica. Elegimos esta porque nos ahorra mucho tiempo de pruebas.

Nota: Aplicaremos la función `log1p` de Numpy a la variable dependiente, ya que Kaggle especifica que medirá el NWRMSLE:

![img/nwrmsle.png](img/nwrmsle.png)

Primero escribimos algunas funciones:

In [6]:
def preprocess_data(data):
    '''
    Función que se encarga de preprocesar los datos de entrada
    '''
    # La columna onpromotion se cargó como object, ya que hay que arreglar los NA. 
    # Sustituimos los NA por False ("sin promoción"), y convertimos la columna en tipo bool.
    data.onpromotion.fillna(False, inplace=True)
    data.onpromotion = data.onpromotion.map({'False': False, 'True': True})
    data.onpromotion = data.onpromotion.astype(bool)
    
    # Usamos `np.clip` para cambiar los valores negativos (debidos a devoluciones) por un 0, 
    # de forma que el argumento mínimo de log1p sea 1 y su resultado mínimo sea 0.
    data.unit_sales = np.log1p(np.clip(data.unit_sales, 0, None))
    
    # Usamos la función add_datepart con el campo date.
    add_datepart(data, 'date')

In [7]:
def process_dataset_in_chunks():
    '''
    Función que carga y preprocesa los datos usando chunks
    '''
    # Leer datos. Cargaremos el dataset especificando el nº de filas por chunk en `chunksize`,
    # dado que tendremos problemas de memoria si trabajamos con el dataset completo.
    file_reader = pd.read_csv('data/train.csv', parse_dates = ['date'], dtype = types, 
                     infer_datetime_format = True, chunksize=10000000)

    chunk_list = [] 

    for chunk in file_reader:

        # Preprocesado del chunk
        preprocess_data(chunk)

        # Agregamos el chunk a la lista
        chunk_list.append(chunk)

    # concat the list into dataframe 
    return pd.concat(chunk_list)

In [8]:
def process_dataset(file_path):
    '''
    Función que carga y preprocesa los datos
    '''
    # Leer datos de un fichero original
    df_sample = pd.read_csv(file_path, parse_dates = ['date'], dtype = types, 
                            infer_datetime_format = True)

    # Preprocesado
    preprocess_data(df_sample)
    
    return df_sample

A continuación ejecutamos lo necesario para seguir la estrategia elegida. Primero creamos un dataset con las últimas N filas del original:

In [9]:
!head -1 data/train.csv > data/train_sample.csv
!tail -10000000 data/train.csv >> data/train_sample.csv
#!cat data/train_sample.csv

In [10]:
df_all = process_dataset('data/train_sample.csv')

In [11]:
%time df_all.describe(include='all')

Wall time: 8.92 s


,id,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
count,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,10000000,10000000.0,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,10000000,10000000,10000000,10000000,10000000,10000000,1.000000e+07
unique,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,1,1,NaN
top,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,NaN
freq,NaN,NaN,NaN,NaN,8791824,NaN,NaN,NaN,NaN,NaN,NaN,9682005,9666352,9894888,9881806,10000000,10000000,NaN
mean,1.204970e+08,2.833271e+01,1.169230e+06,1.692971e+00,NaN,2017.0,6.444426e+00,2.599360e+01,1.571219e+01,3.048752e+00,1.800040e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.498695e+09
std,2.886751e+06,1.630415e+01,5.861997e+05,8.704344e-01,NaN,0.0,9.784031e-01,3.916341e+00,8.781373e+00,2.046676e+00,2.730393e+01,NaN,NaN,NaN,NaN,NaN,NaN,2.359059e+06
min,1.154970e+08,1.000000e+00,9.699500e+04,0.000000e+00,NaN,2017.0,5.000000e+00,1.900000e+01,1.000000e+00,0.000000e+00,1.330000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.494634e+09
25%,1.179970e+08,1.300000e+01,6.919450e+05,1.098612e+00,NaN,2017.0,6.000000e+00,2.300000e+01,8.000000e+00,1.000000e+00,1.560000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.496621e+09
50%,1.204970e+08,2.900000e+01,1.209720e+06,1.609438e+00,NaN,2017.0,6.000000e+00,2.600000e+01,1.500000e+01,3.000000e+00,1.800000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.498694e+09
75%,1.229970e+08,4.400000e+01,1.576316e+06,2.197225e+00,NaN,2017.0,7.000000e+00,2.900000e+01,2.300000e+01,5.000000e+00,2.040000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.500768e+09


Guardamos el dataframe en formato feather para no tener que repetir la carga:

In [12]:
os.makedirs('tmp', exist_ok=True)
%time df_all.to_feather('tmp/raw_grocery')

Wall time: 909 ms


# Muestras de entrenamiento y validación

In [13]:
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()

n_valid = 2000000  # las mismas muestras que los datasets de test
n_trn = len(df_all) - n_valid

train, valid = split_vals(df_all, n_trn)

train.shape, valid.shape

((8000000, 18), (2000000, 18))

In [14]:
X_train, y_train, nas = proc_df(train, 'unit_sales')
X_valid, y_valid, _ = proc_df(valid, 'unit_sales')

# Entrenamiento y validación del modelo

In [15]:
def rmse(predictions, actuals): 
    return math.sqrt(((predictions - actuals)**2).mean())

def print_score(m):
    print('RMSE for training:   ', rmse(m.predict(X_train), y_train))
    print('RMSE for validation: ', rmse(m.predict(X_valid), y_valid))
    print('R^2 for training:    ', m.score(X_train, y_train))
    print('R^2 for validation:  ', m.score(X_valid, y_valid))

In [16]:
set_rf_samples(1_000_000)

Pasamos X a un array de Numpy para que cada vez que entrenemos el modelo con distintos parámetros no se haga de forma interna:

In [17]:
X_train = np.array(X_train, dtype=np.float32)

Creamos el modelo y lo entrenamos:

In [18]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, n_jobs=-1)
%prun m.fit(X_train, y_train)
print_score(m)

 RMSE for training:    0.5673802075772398
RMSE for validation:  0.6543076402230862
R^2 for training:     0.5756541678226083
R^2 for validation:   0.4320269405678712


Los resultados obtenidos no son del todo malos; de hecho superan a los obtenidos por Jeremy Howard en el curso de fast.ai. Aún así queda pendiente el intentar trabajar con todo el dataset.